# 2.06 - Defining an Objective Function
This workbook takes us through the formulation of an objective function to help indicate the 'goodness' of a night, in terms of glycaemic control, where the night is defined as the period between 22:00 and 06:00 the following day. To recap, the following measures have been studied and will contribute as components to the objective function. These are recorded at each 30 minutes interval in our dataset:
- Blood Glucose Mean
- Blood Glucose Standard Deviation
- Coefficient of Variation
- Intervals Outside Level 1 Target Range
- Intervals Outside Level 2 Target Range
- Amplitude of Glycaemic Variability (absolute value above 1 SD from the mean)
- Peaks of Carbohydrate Intake

The goal is still to produce a continuous variable where the score indicates the quality of the night, encompassing the physiological impact observed through glycaemic metrics and carbohydrate intake, with a higher score indicating greater disturbance.

For each night (22:00 to 06:00), we'll aggregate your 30-minute interval data for the following metrics:
- Blood Glucose Mean (BG_Mean_Night): The average of all 30-minute BG means throughout the night. A higher average BG suggests potential discomfort or dysregulation.
- Blood Glucose Standard Deviation (BG_SD_Night): The average of all 30-minute BG standard deviations (or the overall standard deviation of all BG readings during the night). High variability can indicate instability and potential physiological stress.
- Coefficient of Variation (CV_Night): The average of the 30-minute Coefficient of Variation values. This normalizes variability to the mean, providing a robust measure of relative glucose fluctuations, which can be disturbing.
- Intervals Outside Level 1 Target Range (L1_Excursions_Night): The sum or count of 30-minute intervals where a Level 1 (clinically defined) excursion occurred. These indicate deviations from a generally acceptable range, potentially causing discomfort or impacting restorative processes.
- Intervals Outside Level 2 Target Range (L2_Excursions_Night): The sum or count of 30-minute intervals where a Level 2 (more severe) excursion occurred. These represent more significant deviations, very likely disruptive.
- Amplitude of Glycaemic Variability (AGV_Night): The sum or average of the absolute amplitude values (above 1 SD from the mean). These capture significant, rapid swings in glucose, which are often associated with physiological stress or symptoms that could disturb sleep, even if within clinical thresholds. If the dataset is very sparse for this, consider summing non-zero amplitudes or taking the maximum amplitude for the night to highlight the presence of any large swings.
Important Note: Peaks of Carbohydrate Intake will not be part of this objective function. Instead, it will serve as an explanatory variable that you will later use to understand what contributes to the "Nocturnal Disturbance" score. This keeps the objective function focused on the outcome (disturbance) rather than its potential causes.



In [5]:
from src.features import FeatureSet
from src.nights import Nights, consolidate_df_from_nights
from src.dba import dba_by_cluster
from sklearn.preprocessing import StandardScaler
from src.sample_filter import SampleFilter
from src.configurations import ThirtyMinute
from datetime import time
import pandas as pd
from src.config import PROCESSED_DATA_DIR

df_all = pd.read_parquet(PROCESSED_DATA_DIR / 'night_clusters.parquet')
df_all['time'] = df_all.index.get_level_values('datetime').time

sample_rate = 30
nights_objects = []

night_start = time(22, 0)
morning_end = time(6, 0)

for zip_id, df_ind in df_all.groupby('id'):
    df_ind_reset = df_ind.reset_index(level='id', drop=True)
    nights_objects.append(
        Nights(df=df_ind_reset, zip_id=zip_id, night_start=night_start,
               morning_end=morning_end, sample_rate=sample_rate))

df_overnight = consolidate_df_from_nights(nights_objects)

In [6]:
df_overnight.head()

night_start_date     bg mean  bg min  bg max  \
id       datetime                                                           
15558575 2018-04-17 22:00:00       2018-04-17  237.000000   226.0   248.0   
         2018-04-17 22:30:00       2018-04-17  191.000000   191.0   191.0   
         2018-04-17 23:00:00       2018-04-17  167.000000   160.0   179.0   
         2018-04-17 23:30:00       2018-04-17  146.000000   129.0   160.0   
         2018-04-18 00:00:00       2018-04-17  121.000000   120.0   122.0   
...                                       ...         ...     ...     ...   
97417885 2018-02-18 03:30:00       2018-02-17  226.800003   226.0   227.0   
         2018-02-18 04:00:00       2018-02-17  215.570999   211.0   222.0   
         2018-02-18 04:30:00       2018-02-17  212.332993   210.0   214.0   
         2018-02-18 05:00:00       2018-02-17  158.500000   125.0   200.0   
         2018-02-18 05:30:00       2018-02-17  134.000000   129.0   139.0   

                              iob mean  iob max  cob mean  cob max  cluster  \
id       datetime                                                             
15558575 2018-04-17 22:00:00     4.250    4.686       0.0      0.0        2   
         2018-04-17 22:30:00     1.905    1.905       0.0      0.0        2   
         2018-04-17 23:00:00     1.272    1.560       0.0      0.0        2   
         2018-04-17 23:30:00     0.656    0.960       0.0      0.0        2   
         2018-04-18 00:00:00    -0.298   -0.275       0.0      0.0        2   
...                                ...      ...       ...      ...      ...   
97417885 2018-02-18 03:30:00     0.853    0.897       0.0      0.0        1   
         2018-02-18 04:00:00     0.737    0.857       0.0      0.0        1   
         2018-02-18 04:30:00     0.693    0.709       0.0      0.0        1   
         2018-02-18 05:00:00     0.358    0.465       0.0      0.0        1   
         2018-02-18 05:30:00     0.115    0.122       0.0      0.0        1   

                                 bg std      time  
id       datetime                                  
15558575 2018-04-17 22:00:00     15.556  22:00:00  
         2018-04-17 22:30:00       <NA>  22:30:00  
         2018-04-17 23:00:00      6.272  23:00:00  
         2018-04-17 23:30:00     15.716  23:30:00  
         2018-04-18 00:00:00      1.414  00:00:00  
...                                 ...       ...  
97417885 2018-02-18 03:30:00      0.447  03:30:00  
         2018-02-18 04:00:00      3.359  04:00:00  
         2018-02-18 04:30:00      2.082  04:30:00  
         2018-02-18 05:00:00  38.249001  05:00:00  
         2018-02-18 05:30:00      7.071  05:30:00  

[14605 rows x 11 columns]

In [ ]:
features = FeatureSet()